In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import kdephys as kde
import warnings
import pingouin as pg
from acr.utils import SOM_BLUE, ACR_BLUE, LASER_BLUE, NNXR_GRAY, NNXO_BLUE, EMG_SLATE, BACKUP_RED, PAPER_FIGURE_ROOT
from kdephys.utils.main import td
acr.plots.lrg()
#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

#--------------------------------- PLOTTING STUFF ---------------------------------#
plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle')
probe_ord = ['NNXr', 'NNXo']


#--------------------------------- MISCELLANEOUS ---------------------------------#
#warnings.filterwarnings('ignore')

# SOM SUBJECTS

In [ ]:
subject = 'ACR_25'
exp = 'swi'
recs = acr.info_pipeline.get_exp_recs(subject, exp)
fp = acr.io.load_concat_raw_data(subject, recs)

In [ ]:
h = acr.io.load_hypno_full_exp(subject, exp)
hd = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp)
reb_start = hd['rebound']['start_time'].min()
sd_start, stim_start, stim_end, reb_start, full_x_start = acr.info_pipeline.get_sd_exp_landmarks(subject, exp, update=False, return_early=False)

cbl_start = hd['circ_bl']['start_time'].min()
ebl_start = hd['early_bl']['start_time'].min()

In [ ]:
for bnum, bout in enumerate(hd['rebound'].iloc[0:3].itertuples()):
    dur = bout.duration.total_seconds()
    for i in np.arange(0, dur, 4):
        t1 = bout.start_time + td(i)
        t2 = t1 + td(4)
        f, ax = plt.subplots(1, 1, figsize=(16, 5))
        fp.ts(t1, t2).ch(10).prb('NNXo').plot(ax=ax, color=SOM_BLUE)
        fp.ts(t1, t2).ch(10).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
        ax.set_title(f'{i} sec, from {bout.start_time} -- bout #{bnum}, --- {ax.get_ylim()}', fontsize=12)
        #ax.set_ylim(-1300, 1000)

In [ ]:
bout_num = 0
start_time = hd['rebound'].iloc[bout_num].start_time
offset = 274.9
for chan in np.arange(1, 17):
    t1 = start_time + td(offset)
    t2 = t1 + td(4)
    f, ax = plt.subplots(1, 1, figsize=(16, 5))
    fp.ts(t1, t2).ch(chan).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
    fp.ts(t1, t2).ch(chan).prb('NNXo').plot(ax=ax, color=SOM_BLUE)
    ax.set_title(f'CHAN-{chan} --- {offset} sec, from {start_time} --- {ax.get_ylim()}', fontsize=12)

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = reb_start + td(274.9)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(10).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(10).prb('NNXo').plot(ax=ax, color=SOM_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-800, 950)
print(ax.get_ylim())
ax.set_xlim(times[0], times[-1])
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_REBOUND_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_REBOUND_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hd['early_bl'].iloc[2].start_time + td(357)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(10).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(10).prb('NNXo').plot(ax=ax, color=SOM_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-800, 950)

ax.set_xlim(times[0], times[-1])
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

# ACR SUBJECTS

In [ ]:
subject = 'ACR_20'
exp = 'swi'
recs = acr.info_pipeline.get_exp_recs(subject, exp)

In [ ]:
fp = acr.io.load_concat_raw_data(subject, recs[0:3])
h = acr.io.load_hypno_full_exp(subject, exp)
hd = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp)
reb_start = hd['rebound']['start_time'].min()
sd_start, stim_start, stim_end, reb_start, full_x_start = acr.info_pipeline.get_sd_exp_landmarks(subject, exp, update=False, return_early=False)

cbl_start = hd['circ_bl']['start_time'].min()
ebl_start = hd['early_bl']['start_time'].min()

In [ ]:
bout_num = 0
start_time = hd['early_bl'].iloc[bout_num].start_time
offset = 30
for chan in np.arange(1, 17):
    t1 = start_time + td(offset)
    t2 = t1 + td(4)
    f, ax = plt.subplots(1, 1, figsize=(16, 5))
    fp.ts(t1, t2).ch(chan).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
    fp.ts(t1, t2).ch(chan).prb('NNXo').plot(ax=ax, color=ACR_BLUE)
    ax.set_title(f'CHAN-{chan} --- {offset} sec, from {start_time} --- {ax.get_ylim()}', fontsize=12)

In [ ]:
for bnum, bout in enumerate(hd['rebound'].iloc[0:4].itertuples()):
    dur = bout.duration.total_seconds()
    for i in np.arange(0, dur, 4):
        t1 = bout.start_time + td(i)
        t2 = t1 + td(4)
        f, ax = plt.subplots(1, 1, figsize=(16, 5))
        fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)
        fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
        ax.set_title(f'{i} sec, from {bout.start_time} -- bout #{bnum}, --- {ax.get_ylim()}', fontsize=12)
        #ax.set_ylim(-1300, 1000)

In [ ]:
hypno = hd['rebound']
for bnum, bout in enumerate(hypno.iloc[0:20].itertuples()):
    dur = bout.duration.total_seconds()
    for i in np.arange(0, dur, 4):
        t1 = bout.start_time + td(i)
        t2 = t1 + td(4)
        f, ax = plt.subplots(1, 1, figsize=(16, 5))
        fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)
        fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
        ax.set_title(f'{i} sec, from {bout.start_time} -- bout #{bnum}, --- {ax.get_ylim()}', fontsize=12)
        ax.set_ylim(-750, 1200)
        #ax.set_ylim(-1300, 1000)

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hypno.iloc[1].start_time + td(15.8)
#t1 = reb_start + td(0.2)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-750, 1200)
print(ax.get_ylim())
ax.set_xlim(times[0], times[-1])
#f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_newREBOUND_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
#f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_newREBOUND_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hypno.iloc[1].start_time + td(15.85)
#t1 = reb_start + td(0.2)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-750, 1200)
print(ax.get_ylim())
ax.set_xlim(times[0], times[-1])
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_new2REBOUND_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_new2REBOUND_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hypno.iloc[2].start_time + td(116)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-750, 1200)
print(ax.get_ylim())
ax.set_xlim(times[0], times[-1])

f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hypno.iloc[1].start_time + td(187)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=ACR_BLUE)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-750, 1200)
print(ax.get_ylim())
ax.set_xlim(times[0], times[-1])

f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP2.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP2_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

# Halo Subs

In [ ]:
subject = 'ACR_50'
exp = 'swisin2'
recs = acr.info_pipeline.get_exp_recs(subject, exp)
fp = acr.io.load_concat_raw_data(subject, recs)

In [ ]:
h = acr.io.load_hypno_full_exp(subject, exp)
hd = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp)
reb_start = hd['rebound']['start_time'].min()
sd_start, stim_start, stim_end, reb_start, full_x_start = acr.info_pipeline.get_sd_exp_landmarks(subject, exp, update=False, return_early=False)

cbl_start = hd['circ_bl']['start_time'].min()
ebl_start = hd['early_bl']['start_time'].min()

In [ ]:
for bnum, bout in enumerate(hd['early_bl'].iloc[7:12].itertuples()):
    dur = bout.duration.total_seconds()
    for i in np.arange(0, dur, 4):
        t1 = bout.start_time + td(i)
        t2 = t1 + td(4)
        f, ax = plt.subplots(1, 1, figsize=(16, 5))
        fp.ts(t1, t2).ch(9).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
        fp.ts(t1, t2).ch(9).prb('NNXo').plot(ax=ax, color=HALO_GREEN, alpha=0.9)
        ax.set_title(f'{i} sec, from {bout.start_time} -- bout #{bnum}, --- {ax.get_ylim()}', fontsize=12)
        ax.set_ylim(-1000, 1000)

In [ ]:
bout_num = 0
start_time = hd['rebound'].iloc[bout_num].start_time
offset = 274.9
for chan in np.arange(1, 17):
    t1 = start_time + td(offset)
    t2 = t1 + td(4)
    f, ax = plt.subplots(1, 1, figsize=(16, 5))
    fp.ts(t1, t2).ch(chan).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
    fp.ts(t1, t2).ch(chan).prb('NNXo').plot(ax=ax, color=SOM_BLUE)
    ax.set_title(f'CHAN-{chan} --- {offset} sec, from {start_time} --- {ax.get_ylim()}', fontsize=12)

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hd['rebound'].iloc[1].start_time + td(123.5)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(10).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(10).prb('NNXo').plot(ax=ax, color=HALO_GREEN, alpha=0.9)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
print(ax.get_ylim())
ax.set_ylim(-800, 750)
ax.set_xlim(times[0], times[-1])
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_REBOUND_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_REBOUND_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())

In [ ]:
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['xtick.bottom'] = False
t1 = hd['early_bl'].iloc[10].start_time + td(28)
t2 = t1 + td(4)
times = [t1+td(i) for i in np.arange(0, 5)]
f, ax = plt.subplots(1, 1, figsize=(12, 3))
fp.ts(t1, t2).ch(10).prb('NNXr').plot(ax=ax, color=NNXR_GRAY)
fp.ts(t1, t2).ch(10).prb('NNXo').plot(ax=ax, color=HALO_GREEN, alpha=0.9)

ax.set_title('')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_yticks([])
ax.set_xticks(times)
plt.subplots_adjust(wspace=0.001)
ax.set_ylim(-800, 750)

ax.set_xlim(times[0], times[-1])
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP.png', dpi=600, bbox_inches='tight', transparent=True)

#labelled plot
ax.axhspan(-250, 250, color='red', alpha=0.2)
val = ax.get_ylim()[1]-200
ax.plot([times[1], times[2]], [val, val], color='green', lw=3)
f.savefig(f'{PAPER_FIGURE_ROOT}/trace_snippets/{subject}_{exp}_BL_TRACE_SNIP_labeled--500uV--1sec.png', dpi=600, bbox_inches='tight', transparent=True)
print(ax.get_ylim())